In [4]:
import json
import gzip
import os
from datetime import timedelta, date

In [6]:
from paper.config import *

by_dir

'/Users/oyvsyo/proj/jooble/IR/jooble-ir-2018-paper/Data/by'

In [25]:
list_json = []

start_date = date(2017, 1, 1)
# end_date = date(2017, 1, 30)
end_date = date(2017, 1, 1)

date_range = pd.date_range(start_date, end_date)
    
#looping through all days from 2017_01_1 to 2017_01_30 with step = 1 day
for single_date in date_range:
    date_ = single_date.strftime("%Y-%m-%d")
        
    #!!!!CHANGE PATH!!!!!!
    file_job = os.path.join(by_dir, 'by_job_{}.json.gz'.format(date_))
    file_job_text = os.path.join(by_dir, 'by_job_text_{}.json.gz'.format(date_))

    #reading two files simuntaneously
    document_job, document_job_text = [],[]
    with gzip.open(file_job, "rb") as f1, gzip.open(file_job_text, "rb") as f2: 
        for line1, line2 in zip(f1, f2):
            vacation_job = json.loads(line1)
            vacation_job_text = json.loads(line2)
            document_job.append(vacation_job)
            document_job_text.append(vacation_job_text)

        #creating a list of dictionaries
        for i in range(len(document_job)):
            for j in range(len(document_job_text)):
                if document_job[i]['id'] == document_job_text[j]['id_job']:
                    data = {}
                    data['id'] = document_job[i]['id']
                    data['lang_text'] = document_job[i]['lang_text']
                    data['lang_title'] = document_job[i]['lang_title']
                    data['company_name'] = document_job[i]['company_name']
                    data['title'] = document_job[i]['title']
                    data['title_normalized'] = document_job[i]['title_normalized']
                    data['salary_val1'] = document_job[i]['salary_val1']
                    data['salary_val2'] = document_job[i]['salary_val2']
                    data['company_name'] = document_job[i]['company_name']
                    data['url'] = document_job[i]['url']
                    
                    data['text'] = document_job_text[j]['text']
                    list_json.append(data)
                    
print(len(list_json))

219


## Make Index update via `updater` service

In [29]:
from paper.utils import post_request as post


update_json = {'docs': list_json}

post(UPDATER_PORT, UPDATER_PATH, update_json)

{'ids': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  15